In [ ]:
D = "USpowerGrid"
N = 500

EDGE_WEIGHT = 30


In [ ]:
# Standard Library
import argparse
from time import perf_counter

# Third Party Library
import matplotlib.pyplot as plt
import networkx as nx
import optuna
import pandas as pd
from egraph import Drawing, all_sources_bfs
from ex_utils.config.dataset import dataset_names
from ex_utils.config.paths import get_dataset_path
from ex_utils.config.quality_metrics import qm_names
from ex_utils.share import draw_and_measure, ex_path, generate_base_df_data
from ex_utils.utils.graph import (
    egraph_graph,
    load_nx_graph,
    nx_graph_preprocessing,
)
from tqdm import tqdm


In [ ]:
db_uri = (
    f"sqlite:///{ex_path.joinpath('results/optimization/optimization.db')}"
)
study_name = f"{D}_N={N}_with_runtime"
study = optuna.load_study(study_name=study_name, storage=db_uri)
best_trials = study.best_trials

In [ ]:
best_trials_quality_metrics = dict((qm_name, []) for qm_name in qm_names)
for best_trial in best_trials:
    for qm_name, value in zip(qm_names, best_trial.values):
        best_trials_quality_metrics[qm_name].append(value)

for qm_name in qm_names:
    best_trials_quality_metrics[qm_name] = sorted(
        best_trials_quality_metrics[qm_name]
    )

for best_trial in best_trials:
    for qm_name, value in zip(qm_names, best_trial.values):
        best_trial.set_user_attr(
            f"{qm_name}_asc_index",
            best_trials_quality_metrics[qm_name].index(value),
        )
    s = 0
    for qm_name in qm_names:
        s += best_trial.user_attrs[f"{qm_name}_asc_index"]
    best_trial.set_user_attr("asc_index_total", s)


In [ ]:
list_best_trials = []
for best_trial in best_trials:
    item = {
        "params": best_trial.params,
        "values": best_trial.values,
        **best_trial.user_attrs,
    }

    list_best_trials.append(item)

desc_sorted_list_best_trials = sorted(
    list_best_trials, key=lambda x: x["asc_index_total"], reverse=True
)


In [ ]:
dataset_path = get_dataset_path(D)
nx_graph = nx_graph_preprocessing(
    load_nx_graph(dataset_path=dataset_path), EDGE_WEIGHT
)

eg_graph, eg_indices = egraph_graph(nx_graph=nx_graph)
eg_distance_matrix = all_sources_bfs(eg_graph, EDGE_WEIGHT)


for t in desc_sorted_list_best_trials:
    pivots = t["params"]["pivots"]
    iterations = t["params"]["iterations"]
    eps = t["params"]["eps"]

    eg_drawing = Drawing.initial_placement(eg_graph)

    params, quality_metrics, pos = draw_and_measure(
        pivots=pivots,
        iterations=iterations,
        eps=eps,
        eg_graph=eg_graph,
        eg_indices=eg_indices,
        eg_drawing=eg_drawing,
        eg_distance_matrix=eg_distance_matrix,
        edge_weight=EDGE_WEIGHT,
        seed=0,
    )

    fig, ax = plt.subplots(dpi=300, facecolor="white")
    ax.set_aspect("equal")

    ax.set_title(
        f"""{D} pivots={pivots},iter={iterations},eps={eps}
best_n={len(best_trials)},asc_index_total={t['asc_index_total']}
{'''
'''.join([f'{qm_name}={t[f"{qm_name}_asc_index"]}' for qm_name in qm_names])}"""
    )

    nx.draw(
        nx_graph,
        pos=pos,
        node_size=5,
        node_color="#AB47BC",
        edge_color="#CFD8DC",
        ax=ax,
    )
    print(pos)

    print(quality_metrics)

    plt.show()